# RTC dataset comparison

Still do to on this:
- discuss differences btw two datasets
- confirm PC doesn't include layover shadow map?
- apply ASF layover-shadow map to PC dataset and compare <-- tricky to line up?

In [12]:
import xarray as xr

import s1_tools

from s1_tools import points2coords

import os
import xarray as xr
import rioxarray as rio
import geopandas as gpd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import holoviews as hv
from holoviews import opts


## Read in PC dataset

In [13]:
from pystac_client import Client

catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
catalog

time_range = '2021-01-01/2022-08-01'
bbox = [88.214935, 27.92767, 88.302,  28.034]

bbox_coords = points2coords(bbox)
bbox_coords

search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

import stackstac
import os
import planetary_computer

time_range = '2021-01-01/2022-08-01'
bbox = [88.214935, 27.92767, 88.302,  28.034]

bbox_coords = points2coords(bbox)
bbox_coords

#print(os.environ.get('PC_SDK_SUBSCRIPTION_KEY'))
#bounds_latlon=bbox,
da_pc = stackstac.stack(
    planetary_computer.sign(items), bounds_latlon=bbox, epsg=32645, #resolution = 50
)
da_pc

,Array,Chunk
Bytes,1.54 GiB,6.79 MiB
Shape,"(100, 2, 1188, 869)","(1, 1, 1024, 869)"
Count,3 Graph Layers,400 Chunks
Type,float64,numpy.ndarray


## Read in ASF dataset

We can use the `storemagic` command `%store` to retrieve the variable we constructed and saved in a previous notebook, rather than having to create it again. Read more about this [here](https://levelup.gitconnected.com/how-to-store-variables-in-jupyter-notebook-fea8aa60a9b)

In [14]:
%store -r

In [4]:
asf_clip

<xarray.Dataset>
Dimensions:      (x: 290, y: 396, acq_date: 103)
Coordinates:
  * x            (x) float64 6.194e+05 6.195e+05 ... 6.281e+05 6.281e+05
  * y            (y) float64 3.09e+06 3.09e+06 3.09e+06 ... 3.102e+06 3.102e+06
  * acq_date     (acq_date) datetime64[ns] 2021-05-02 2021-05-05 ... 2022-05-21
    band         int64 1
    spatial_ref  int64 0
Data variables:
    vv           (acq_date, y, x) float32 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>
    vh           (acq_date, y, x) float32 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>
    ls           (acq_date, y, x) float32 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>

In [15]:
vrt_full

<xarray.Dataset>
Dimensions:                           (time: 103, x: 290, y: 396, sensor: 103,
                                       beam_mode: 103, acquisition_date: 103,
                                       acquisition_time: 103,
                                       polarisation_type: 103, orbit_type: 103,
                                       terrain_correction_pixel_spacing: 103,
                                       output_format: 103, output_type: 103,
                                       masked: 103, filtered: 103, area: 103,
                                       product_id: 103, acq_date: 103)
Coordinates: (12/19)
  * time                              (time) datetime64[ns] 2021-05-02 ... 20...
  * x                                 (x) float64 6.194e+05 ... 6.281e+05
  * y                                 (y) float64 3.102e+06 ... 3.09e+06
    spatial_ref                       int64 0
  * sensor                            (sensor) <U3 'S1A' 'S1A' ... 'S1A' 'S1A'
  * beam_mode                         (beam_mode) <U2 'IW' 'IW' ... 'IW' 'IW'
    ...                                ...
  * masked                            (masked) <U1 'u' 'u' 'u' ... 'u' 'u' 'u'
  * filtered                          (filtered) <U1 'n' 'n' 'n' ... 'n' 'n' 'n'
  * area                              (area) <U1 'e' 'e' 'e' 'e' ... 'e' 'e' 'e'
  * product_id                        (product_id) <U4 '748F' '0D1E' ... 'BD36'
  * acq_date                          (acq_date) datetime64[ns] 2021-08-21 .....
    granule_id                        (acq_date) <U67 'S1A_IW_SLC__1SDV_20210...
Data variables:
    vv                                (time, y, x) float32 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>
    vh                                (time, y, x) float32 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>
    ls                                (time, y, x) uint8 dask.array<chunksize=(1, 396, 290), meta=np.ndarray>

## Dataset comparison

Before we start, let's think about the different aspects of the two datasets that we want to compare, and how we can accomplish this using xarray. Comparing a single time step can be informative, but it doesn't give us a representative idea of how the distributions of the two datasets may be similar or dissimilar. For that, we'd like to perform a **reduction** that let's us view a 

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
power_to_db(asf_clip.vv.mean(dim=['x','y'])).plot(ax=ax, color = 'blue', linestyle='None', marker='o', markersize=5)
power_to_db(da_pc.sel(band='vv').mean(dim=['x','y'])).plot(ax=ax, color='red', linestyle='None', marker='o', markersize=5)